# Operations development

SimPhoNy operations are actions (written in Python) that can be executed on
demand on any ontology individual belonging to the ontology class they
are defined for.

<div class="admonition important">
<div class="admonition-title" style="font-weight: bold"><div style="display: inline-block">Tip</div></div>
    
File uploads and downloads in SimPhoNy are an example of SimPhoNy operations.
Head to the [assertional knowledge](../usage/assertional_knowledge.html#Operations) section to see them in action.
    
</div>


SimPhoNy operations are distributed as (or as part of) Python packages. 
Developing operations for an ontology class is fairly simple. To do so, import the [Operations abstract class](../api_reference.md#simphony_osp.development.Operations) from the `simphony_osp.development` module, and create an implementation by subclassing it.

Then, define an `iri` attribute having as value the IRI of the ontology class that the
operation should be associated to. Every public method ([not starting with
an underscore](https://peps.python.org/pep-0008/#method-names-and-instance-variables)) defined on the implementation is automatically detected and
assigned as an operation to said ontology class. The
[ontology individual object](../usage/assertional_knowledge.ipynb#Ontology-individual-objects)
on which the operation is called is available as the private attribute
`_individual` on every instance of the implementation. For a specific 
ontology individual, the implementation gets instantiated the first time that 
any operation defined on it is called by the user.

Finally, define an
[entry point](https://packaging.python.org/en/latest/specifications/entry-points/#entry-points-specification)
(or many if implementing operations for several ontology classes) under the
`simphony_osp.ontology.operations`
[group](https://packaging.python.org/en/latest/specifications/entry-points/#data-model)
that points to your
implementation of the `Operations` abstract class.

An example implementation of an operation that takes advantage of
[geopy](https://pypi.org/project/geopy/) to compute the distance between 
two points on Earth defined using the 
[WGS84 Geo Positioning vocabulary](https://www.w3.org/2003/01/geo/wgs84_pos)
is shown below.

```python
"""Operations for classes from the WGS84 Geo Positioning vocabulary."""

from geopy import distance
from simphony_osp.namespaces import wgs84_pos
from simphony_osp.ontology import OntologyIndividual
from simphony_osp.ontology.operations import Operations


class Wgs84Pos(Operations):
    """Operations for the Point ontology class."""

    iri = wgs84_pos.Point.iri

    def distance(self, other: OntologyIndividual) -> float:
        """Compute the distance between two points on Earth.
        
        Args:
            other: Another point with respect to which the distance will be computed. 
            
        Returns:
            The distance between this point and the given one in km.
        """
        lat_self = float(self._individual[wgs84_pos.latitude].one())
        long_self = float(self._individual[wgs84_pos.longitude].one())
        lat_other = float(other[wgs84_pos.latitude].one())
        long_other = float(other[wgs84_pos.longitude].one())
        return distance.geodesic(
            (lat_self, long_self),
            (lat_other, long_other),
            ellipsoid="WGS-84"
        ).km

```

<div class="admonition note">
<div class="admonition-title" style="font-weight: bold"><div style="display: inline-block">Note</div></div>
    
Remember that the implementation above is still not enough for the operation to
work: the corresponding 
[entry point](https://packaging.python.org/en/latest/specifications/entry-points/#entry-points-specification)
for `Wgs84Pos` must have been defined and the `wgs84_pos` vocabulary needs to be installed using [pico](../usage/ontologies/pico.html).
    
</div>

The operation can be used as follows.

In [1]:
from simphony_osp.namespaces import wgs84_pos

location_freiburg = wgs84_pos.Point()
location_freiburg[wgs84_pos.latitude] = 47.997791
location_freiburg[wgs84_pos.longitude] = 7.842609

location_paris = wgs84_pos.Point()
location_paris[wgs84_pos.latitude] = 48.85333
location_paris[wgs84_pos.longitude] = 2.34885

location_freiburg.operations.distance(location_paris)

417.4695920611045